In [28]:
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import requests

In [2]:
def get_soup(url):

	result = requests.get(url)
	content = result.content
	soup = BeautifulSoup(content, 'lxml')
	return soup

In [3]:
teams = ['ATL', 'BOS', 'BRK', 'CHI', 'CHO', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']
i = 0
urls = []
for i in range(0, 10000, 100):
    url = 'https://www.basketball-reference.com/play-index/tgl_finder.cgi?request=1&match=game&lg_id=NBA&team_seed_cmp=eq&opp_seed_cmp=eq&year_min=1984&year_max=2018&is_range=N&game_num_type=team&game_location=H&team_id=LAL&is_overtime=N&order_by=diff_off_rtg&order_by_asc=&offset=' + str(i)
    urls.append(url)

In [5]:
start_url = 'https://www.basketball-reference.com/play-index/tgl_finder.cgi?request=1&player=&match=game&lg_id=NBA&year_min=1984&year_max=2018&team_id=LAL&opp_id=&is_range=N&is_playoffs=&round_id=&best_of=&team_seed=&opp_seed=&team_seed_cmp=eq&opp_seed_cmp=eq&game_num_type=team&game_num_min=&game_num_max=&game_month=&game_location=H&game_result=&is_overtime=&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=diff_off_rtg&order_by_asc=&offset=0'
soup = get_soup(start_url)
df = pd.read_html(str(soup.find('table')))[0]
col = []
for i in range(len(df.columns)):
    col.append(df.columns[i][1])
col.pop(3)

'Unnamed: 3_level_1'

In [6]:
count = 0 
LAL = pd.DataFrame(columns=col)

In [7]:
for url in urls:
    try:
        soup = get_soup(url)
        df = pd.read_html(str(soup.find('table')))[0]
        df.drop('Opponent',axis=1, inplace = True)
        df.reset_index(inplace=True, drop=True)
        print("page" + str(count))
        dftemp = df
        df.columns = col
        #print(df.head())
        for i in range(0,df['Rk'].count()):
            if (dftemp.iloc[i,0] == 'Rk'):
                df = df.drop(df.index[i])
        count += 1
        LAL = LAL.append(df)
    except:
        print ("Unexpected error:")
        #raise
        break

C:\Users\Nikhil\AppData\Local\conda\conda\envs\MachineLearningTutorials\lib\site-packages\pandas\core\generic.py:3108: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


page0
page1
page2
page3
page4
page5
page6
page7
page8
page9
page10
page11
page12
page13
page14
page15
Unexpected error:


ValueError: No tables found

In [8]:
len(LAL)

1535

In [9]:
teamdf = pd.DataFrame(columns = col)
df = df.reset_index(drop=True)

In [10]:
clean_df = LAL[LAL.Rk != 'Rk']

In [11]:
clean_df=clean_df.reset_index(drop=True)

In [12]:
clean_df['R'] = 0
clean_df['PTS'] = 0
clean_df['PTSA']= 0

In [13]:
clean_df.loc[i,'Result']

'W 112-82'

In [14]:
clean_df = clean_df.reset_index(drop=True)
for i in range(len(clean_df)):
    r, points = (clean_df.loc[i,'Result']).split()
    PTS, PTSA = points.split('-')
    clean_df.loc[[i],['R']], clean_df.loc[[i],['PTS']], clean_df.loc[[i],['PTSA']] = r, PTS, PTSA

clean_df.drop(['Result'], axis=1, inplace= True)

In [15]:
ad = clean_df.iloc[:, [0,1,2,3,4,-9,-7,-6,-5,-4,-3,-2,-1]]

In [81]:
clean_df.sort_values(by = 'Date', inplace = True)
clean_df = clean_df.reset_index(drop=True)
clean_df.head()

,Rk,Date,Tm,Opp,MP,ORtg,FTr,3PAr,TS%,eFG%,...,FT/FGA,ORtg,FTr,3PAr,TS%,eFG%,FT/FGA,R,PTS,PTSA
0,477,1983-11-09,LAL,DAL,240.0,114.1,.298,0.032,0.564,0.543,...,0.458,13.3,-.256,0.008,0.051,0.133,-0.266,1.0,120.0,106.0
1,478,1983-11-11,LAL,PHO,240.0,113.2,.407,0.047,0.587,0.552,...,0.167,13.3,+.126,0.005,0.100,0.089,0.112,1.0,119.0,105.0
2,880,1983-11-13,LAL,UTA,240.0,117.0,.336,0.019,0.505,0.477,...,0.260,5.7,+.006,0.009,-0.010,0.017,-0.054,1.0,124.0,118.0
3,80,1983-11-15,LAL,MIL,240.0,121.9,.117,0.032,0.637,0.622,...,0.296,28.1,-.241,-0.042,0.120,0.172,-0.201,1.0,126.0,97.0
4,502,1983-11-17,LAL,CLE,240.0,124.8,.435,0.047,0.627,0.588,...,0.133,12.8,+.221,0.027,0.095,0.073,0.185,1.0,127.0,114.0


In [82]:
ad = clean_df
ad = ad.iloc[:, [0,1,2,3,4,-9,-7,-6,-5,-4,-3,-2,-1]]

In [83]:
ad.head()

,Rk,Date,Tm,Opp,MP,ORtg,3PAr,TS%,eFG%,FT/FGA,R,PTS,PTSA
0,477,1983-11-09,LAL,DAL,240.0,13.3,0.008,0.051,0.133,-0.266,1.0,120.0,106.0
1,478,1983-11-11,LAL,PHO,240.0,13.3,0.005,0.100,0.089,0.112,1.0,119.0,105.0
2,880,1983-11-13,LAL,UTA,240.0,5.7,0.009,-0.010,0.017,-0.054,1.0,124.0,118.0
3,80,1983-11-15,LAL,MIL,240.0,28.1,-0.042,0.120,0.172,-0.201,1.0,126.0,97.0
4,502,1983-11-17,LAL,CLE,240.0,12.8,0.027,0.095,0.073,0.185,1.0,127.0,114.0


In [85]:
type(ad["MP"][0])

numpy.float64

In [88]:
ad.head()

,Rk,Date,Tm,Opp,MP,ORtg,3PAr,TS%,eFG%,FT/FGA,R,PTS,PTSA
0,477,1983-11-09,LAL,DAL,240.0,13.3,0.008,0.051,0.133,-0.266,1.0,120.0,106.0
1,478,1983-11-11,LAL,PHO,240.0,13.3,0.005,0.100,0.089,0.112,1.0,119.0,105.0
2,880,1983-11-13,LAL,UTA,240.0,5.7,0.009,-0.010,0.017,-0.054,1.0,124.0,118.0
3,80,1983-11-15,LAL,MIL,240.0,28.1,-0.042,0.120,0.172,-0.201,1.0,126.0,97.0
4,502,1983-11-17,LAL,CLE,240.0,12.8,0.027,0.095,0.073,0.185,1.0,127.0,114.0


In [118]:
ad.to_csv('LAL_advanced.csv', sep = ',')
clean_df.to_csv('LAL_advanced_full.csv', sep = ',')

In [119]:
df = pd.read_csv('LAL_advanced.csv', sep=',', index_col = 0)

In [120]:
df.head()

,Rk,Date,Tm,Opp,MP,ORtg,3PAr,TS%,eFG%,FT/FGA,R,PTS,PTSA
0,477,1983-11-09,LAL,DAL,240.0,13.3,0.008,0.051,0.133,-0.266,1.0,120.0,106.0
1,478,1983-11-11,LAL,PHO,240.0,13.3,0.005,0.100,0.089,0.112,1.0,119.0,105.0
2,880,1983-11-13,LAL,UTA,240.0,5.7,0.009,-0.010,0.017,-0.054,1.0,124.0,118.0
3,80,1983-11-15,LAL,MIL,240.0,28.1,-0.042,0.120,0.172,-0.201,1.0,126.0,97.0
4,502,1983-11-17,LAL,CLE,240.0,12.8,0.027,0.095,0.073,0.185,1.0,127.0,114.0


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1490 entries, 0 to 1489
Data columns (total 13 columns):
Rk        1490 non-null int64
Date      1490 non-null object
Tm        1490 non-null object
Opp       1490 non-null object
MP        1490 non-null float64
ORtg      1489 non-null float64
3PAr      1490 non-null float64
TS%       1490 non-null float64
eFG%      1490 non-null float64
FT/FGA    1490 non-null float64
R         1490 non-null float64
PTS       1490 non-null float64
PTSA      1490 non-null float64
dtypes: float64(9), int64(1), object(3)
memory usage: 163.0+ KB


In [122]:
df.iloc[:,5:].corr()

,ORtg,3PAr,TS%,eFG%,FT/FGA,R,PTS,PTSA
ORtg,1.000000,-0.004346,0.811558,0.782700,0.232105,0.785373,0.552517,-0.461873
3PAr,-0.004346,1.000000,0.065024,0.066807,0.015880,-0.065922,0.001911,0.007434
TS%,0.811558,0.065024,1.000000,0.949396,0.373108,0.673313,0.471149,-0.351833
eFG%,0.782700,0.066807,0.949396,1.000000,0.102764,0.630680,0.454213,-0.340073
FT/FGA,0.232105,0.015880,0.373108,0.102764,1.000000,0.290306,0.124096,-0.111370
R,0.785373,-0.065922,0.673313,0.630680,0.290306,1.000000,0.430828,-0.362047
PTS,0.552517,0.001911,0.471149,0.454213,0.124096,0.430828,1.000000,0.481141
PTSA,-0.461873,0.007434,-0.351833,-0.340073,-0.111370,-0.362047,0.481141,1.000000


In [123]:
clean_df.columns

Index(['Rk', 'Date', 'Tm', 'Opp', 'MP', 'ORtg', 'FTr', '3PAr', 'TS%', 'eFG%',
       'FT/FGA', 'ORtg', 'FTr', '3PAr', 'TS%', 'eFG%', 'FT/FGA', 'ORtg', 'FTr',
       '3PAr', 'TS%', 'eFG%', 'FT/FGA', 'R', 'PTS', 'PTSA'],
      dtype='object')

In [124]:
clean_df.iloc[:,[5,6,7,8,9,10,17,18,19,20,21,22,23,24,25]].corr()

,ORtg,3PAr,TS%,eFG%,FT/FGA,ORtg,3PAr,TS%,eFG%,FT/FGA,R,PTS,PTSA
ORtg,1.000000,-0.189164,0.797078,0.749370,0.226005,0.617694,0.021152,0.491490,0.477327,0.128476,0.493333,0.828594,0.247080
3PAr,-0.189164,1.000000,-0.127097,-0.083252,-0.115980,-0.206719,0.346228,-0.169386,-0.141245,-0.092276,-0.217934,-0.322549,-0.113315
TS%,0.797078,-0.127097,1.000000,0.947891,0.297279,0.555385,0.067801,0.704071,0.672380,0.251268,0.461641,0.767348,0.239716
eFG%,0.749370,-0.083252,0.947891,1.000000,0.022822,0.526619,0.057395,0.660340,0.700394,0.048458,0.425653,0.725247,0.224644
FT/FGA,0.226005,-0.115980,0.297279,0.022822,1.000000,0.164000,0.058339,0.253915,0.079211,0.709338,0.205929,0.204575,0.045990
ORtg,0.617694,-0.206719,0.555385,0.526619,0.164000,1.000000,-0.004346,0.811558,0.782700,0.232105,0.785373,0.552517,-0.461873
3PAr,0.021152,0.346228,0.067801,0.057395,0.058339,-0.004346,1.000000,0.065024,0.066807,0.015880,-0.065922,0.001911,0.007434
TS%,0.491490,-0.169386,0.704071,0.660340,0.253915,0.811558,0.065024,1.000000,0.949396,0.373108,0.673313,0.471149,-0.351833
eFG%,0.477327,-0.141245,0.672380,0.700394,0.079211,0.782700,0.066807,0.949396,1.000000,0.102764,0.630680,0.454213,-0.340073
FT/FGA,0.128476,-0.092276,0.251268,0.048458,0.709338,0.232105,0.015880,0.373108,0.102764,1.000000,0.290306,0.124096,-0.111370
